In [1]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/02 15:48:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/02 15:48:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
spark


In [42]:
##read dataset
df_s=spark.read.csv('test1.csv',header=True,inferSchema=True)
df_s.show()

+-----+----+----------+------+
| name| age|experience|Salary|
+-----+----+----------+------+
|sakai|  19|         1| 10000|
|  sam|  36|        10| 69000|
|  bob|  24|         3| 30000|
|  pam|  38|        14| 74000|
| john|NULL|      NULL| 40000|
| NULL|  34|        10| 45689|
| NULL|  50|        30|100000|
+-----+----+----------+------+



In [43]:
df_s.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [44]:
type(df_s)

pyspark.sql.classic.dataframe.DataFrame

In [45]:
df_s.head(3)

[Row(name='sakai', age=19, experience=1, Salary=10000),
 Row(name='sam', age=36, experience=10, Salary=69000),
 Row(name='bob', age=24, experience=3, Salary=30000)]

In [46]:
df_s.select('name').show()

+-----+
| name|
+-----+
|sakai|
|  sam|
|  bob|
|  pam|
| john|
| NULL|
| NULL|
+-----+



In [47]:
df_s['name']

Column<'name'>

In [48]:
df_s.dtypes

[('name', 'string'), ('age', 'int'), ('experience', 'int'), ('Salary', 'int')]

In [49]:
df_s.describe().show()

25/08/02 16:12:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----+-----------------+------------------+------------------+
|summary|name|              age|        experience|            Salary|
+-------+----+-----------------+------------------+------------------+
|  count|   5|                6|                 6|                 7|
|   mean|NULL|             33.5|11.333333333333334|52669.857142857145|
| stddev|NULL|10.94988584415381|10.347302385968366|30292.154542216438|
|    min| bob|               19|                 1|             10000|
|    max| sam|               50|                30|            100000|
+-------+----+-----------------+------------------+------------------+



In [50]:
##add column to df
df_s=df_s.withColumn('age after 2 years',df_s['age']+2)

In [51]:
#drop column
df_s.drop('age after 2 years').show()

+-----+----+----------+------+
| name| age|experience|Salary|
+-----+----+----------+------+
|sakai|  19|         1| 10000|
|  sam|  36|        10| 69000|
|  bob|  24|         3| 30000|
|  pam|  38|        14| 74000|
| john|NULL|      NULL| 40000|
| NULL|  34|        10| 45689|
| NULL|  50|        30|100000|
+-----+----+----------+------+



In [52]:
#Rename column
df_s.withColumnRenamed('age after 2 years', 'new age').show()

+-----+----+----------+------+-------+
| name| age|experience|Salary|new age|
+-----+----+----------+------+-------+
|sakai|  19|         1| 10000|     21|
|  sam|  36|        10| 69000|     38|
|  bob|  24|         3| 30000|     26|
|  pam|  38|        14| 74000|     40|
| john|NULL|      NULL| 40000|   NULL|
| NULL|  34|        10| 45689|     36|
| NULL|  50|        30|100000|     52|
+-----+----+----------+------+-------+



In [53]:
#HANDLING MISSING VALUES
#MEAN MEDIAN MODE

In [54]:
df_s.na.drop().show()

+-----+---+----------+------+-----------------+
| name|age|experience|Salary|age after 2 years|
+-----+---+----------+------+-----------------+
|sakai| 19|         1| 10000|               21|
|  sam| 36|        10| 69000|               38|
|  bob| 24|         3| 30000|               26|
|  pam| 38|        14| 74000|               40|
+-----+---+----------+------+-----------------+



In [56]:
### any==how
df_s.na.drop(how='any').show()

+-----+---+----------+------+-----------------+
| name|age|experience|Salary|age after 2 years|
+-----+---+----------+------+-----------------+
|sakai| 19|         1| 10000|               21|
|  sam| 36|        10| 69000|               38|
|  bob| 24|         3| 30000|               26|
|  pam| 38|        14| 74000|               40|
+-----+---+----------+------+-----------------+



In [67]:
#threshold
#at least has 4 that arent null
df_s.na.drop(how='any',thresh=4).show()

+-----+---+----------+------+-----------------+
| name|age|experience|Salary|age after 2 years|
+-----+---+----------+------+-----------------+
|sakai| 19|         1| 10000|               21|
|  sam| 36|        10| 69000|               38|
|  bob| 24|         3| 30000|               26|
|  pam| 38|        14| 74000|               40|
| NULL| 34|        10| 45689|               36|
| NULL| 50|        30|100000|               52|
+-----+---+----------+------+-----------------+



In [68]:
df_s.na.drop(how='any',subset=['age']).show()

+-----+---+----------+------+-----------------+
| name|age|experience|Salary|age after 2 years|
+-----+---+----------+------+-----------------+
|sakai| 19|         1| 10000|               21|
|  sam| 36|        10| 69000|               38|
|  bob| 24|         3| 30000|               26|
|  pam| 38|        14| 74000|               40|
| NULL| 34|        10| 45689|               36|
| NULL| 50|        30|100000|               52|
+-----+---+----------+------+-----------------+



In [73]:
df_s.na.fill('Missing Value','name').show()

+-------------+----+----------+------+-----------------+
|         name| age|experience|Salary|age after 2 years|
+-------------+----+----------+------+-----------------+
|        sakai|  19|         1| 10000|               21|
|          sam|  36|        10| 69000|               38|
|          bob|  24|         3| 30000|               26|
|          pam|  38|        14| 74000|               40|
|         john|NULL|      NULL| 40000|             NULL|
|Missing Value|  34|        10| 45689|               36|
|Missing Value|  50|        30|100000|               52|
+-------------+----+----------+------+-----------------+



In [74]:
from pyspark.ml.feature import Imputer
imputer = Imputer(
    inputCols=['age','Experience','Salary'],
    outputCols=["{}_imputed".format(c) for c in ['age','Experience','Salary']]
    ).setStrategy("mean")
#mean can be medioan mode

In [75]:
imputer.fit(df_s).transform(df_s).show()

+-----+----+----------+------+-----------------+-----------+------------------+--------------+
| name| age|experience|Salary|age after 2 years|age_imputed|Experience_imputed|Salary_imputed|
+-----+----+----------+------+-----------------+-----------+------------------+--------------+
|sakai|  19|         1| 10000|               21|         19|                 1|         10000|
|  sam|  36|        10| 69000|               38|         36|                10|         69000|
|  bob|  24|         3| 30000|               26|         24|                 3|         30000|
|  pam|  38|        14| 74000|               40|         38|                14|         74000|
| john|NULL|      NULL| 40000|             NULL|         33|                11|         40000|
| NULL|  34|        10| 45689|               36|         34|                10|         45689|
| NULL|  50|        30|100000|               52|         50|                30|        100000|
+-----+----+----------+------+-----------------+--

In [76]:
#FILTERS

In [80]:
df_s.filter("Salary <=40000").select(['name','age']).show()
#df_s.filter(df_s['Salary'}<-20000).show()
#both are the same and work

+-----+----+
| name| age|
+-----+----+
|sakai|  19|
|  bob|  24|
| john|NULL|
+-----+----+



In [85]:
#~ == not
#& == and
df_s.filter(~(df_s['Salary']<20000) & ~(df_s['age']<20)).show()

+----+---+----------+------+-----------------+
|name|age|experience|Salary|age after 2 years|
+----+---+----------+------+-----------------+
| sam| 36|        10| 69000|               38|
| bob| 24|         3| 30000|               26|
| pam| 38|        14| 74000|               40|
|NULL| 34|        10| 45689|               36|
|NULL| 50|        30|100000|               52|
+----+---+----------+------+-----------------+

